<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [113]:
df =np.array([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [0,1,0],
    [1,0,0],
    [1,1,1],
    [0,0,0]
])

In [145]:
def perceptron(x):
    weight = [[20], [20], [-10]]
    bias = -1.5
    model = np.add(np.dot(x, weight), bias)
    sig = 1/(1+np.exp(-model))
    return np.round(sig)

In [146]:
perceptron(df)

array([[0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.]])

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [2]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [3]:
# input image dimensions
img_rows, img_cols = 28, 28

In [4]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 17s 1us/step


In [5]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [13]:
# Now the data should be in a format you're more familiar with
x_train.shape, x_test.shape

((60000, 784), (10000, 784))

In [11]:
x_train[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [7]:
import numpy as np

y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [2]:
# A Nice Binary target for ya to work with
y_train.shape

NameError: name 'y_train' is not defined

### Estimating Your `net

In [75]:
class NN:
    def __init__(self, iterations = 1000):
        self.niter = iterations
        self.inputs = 784
        self.hiddenNodes = 2
        self.outputNodes = 2
        
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
        
    def sigmoidPrime(self, s):
        return (self.sigmoid(s) * (1-(self.sigmoid(s))))

    def feed_forward(self, X):
        # Weighted sum of inputs => hidden layer
        self.hidden_sum = np.dot(X, self.weights1)

        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)

        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)

        # Final activation of output
        self.activated_output = self.sigmoid(self.output_sum)

        return self.activated_output

    def backward(self, X, y, o):
        # Error in output
        self.o_error = y.reshape(-1,1) - o

       #Error to Sigmoid
        self.o_delta = self.o_error * self.sigmoidPrime(o)

        # z2 error
        self.z2_error = self.o_delta.dot(self.weights2.T)

        # Error explained bi input to hiddenNode
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)

        # Adjustment to first layer
        self.weights1 += X.T.dot(self.z2_delta)
        # Adjustment to second layer
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)


    def train(self, X, y):
        for iter in range(self.niter):
            o = self.feed_forward(X)
            self.backward(X,y,o)


In [76]:
nn = NN()

In [77]:
nn.train(x_train, y_train)

/home/hakujou/anaconda3/envs/u4s1/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in exp
  if sys.path[0] == '':


In [78]:
preds = nn.feed_forward(x_test)

/home/hakujou/anaconda3/envs/u4s1/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in exp
  if sys.path[0] == '':


In [84]:
preds[3][0] > 0

True

In [90]:
new_preds = [1 if i[0] > 0 else 0 for i in preds]
for i in range(len(preds))[:100]:
    print(preds[i], new_preds[i], y_test[i])

[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0.5 0.5] 1 1.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0.5 0.5] 1 0.0
[0. 0.] 0 0.0
[0.5 0.5] 1 1.0
[0.5 0.5] 1 0.0
[0. 0.] 0 0.0
[0.5 0.5] 1 1.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0.5 0.5] 1 1.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0.5 0.5] 1 1.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0.5 0.5] 1 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0.5 0.5] 1 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0.5 0.5] 1 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0.5 0.5] 1 1.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0.5 0.5] 1 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0.5 0.5] 1 0.0
[0. 0.] 0 0.0
[0. 0.] 0 0.0
[0.5 0.5

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?